In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# word2vec 

This notebook is equivalent to `demo-word.sh`, `demo-analogy.sh`, `demo-phrases.sh` and `demo-classes.sh` from Google.

## Training

Download some data, for example: [http://mattmahoney.net/dc/text8.zip](http://mattmahoney.net/dc/text8.zip)

In [4]:
import word2vec

Run `word2phrase` to group up similar words "Los Angeles" to "Los_Angeles"

In [6]:
word2vec.word2phrase('./text8', './text8-phrases', verbose=True)

Starting training using file ./text8
Words processed: 17000K     Vocab size: 4399K  
Vocab size (unigrams + bigrams): 2419827
Words in train file: 17005206


This will create a `text8-phrases` that we can use as a better input for `word2vec`.
Note that you could easily skip this previous step and use the origial data as input for `word2vec`.

Train the model using the `word2phrase` output.

In [8]:
word2vec.word2vec('./text8-phrases', './text8.bin', size=100, verbose=True)

Starting training using file ./text8-phrases
Vocab size: 98331
Words in train file: 15857306
Alpha: 0.000002  Progress: 100.04%  Words/thread/sec: 173.54k  

That generated a `text8.bin` file containing the word vectors in a binary format.

Do the clustering of the vectors based on the trained model.

In [9]:
word2vec.word2clusters('./text8', './text8-clusters.txt', 100, verbose=True)

Starting training using file ./text8
Vocab size: 71291
Words in train file: 16718843
Alpha: 0.000002  Progress: 100.03%  Words/thread/sec: 175.90k  

That created a `text8-clusters.txt` with the cluster for every word in the vocabulary

## Predictions

In [10]:
import word2vec

Import the `word2vec` binary file created above

In [12]:
model = word2vec.load('./text8.bin')

We can take a look at the vocabulaty as a numpy array

In [13]:
model.vocab

array([u'</s>', u'the', u'of', ..., u'denishawn', u'tamiris', u'dolophine'], 
      dtype='<U78')

Or take a look at the whole matrix

In [14]:
model.vectors.shape

(98331, 100)

In [15]:
model.vectors

array([[ 0.14333282,  0.15825513, -0.13715845, ...,  0.05456942,
         0.10955409,  0.00693387],
       [ 0.2372058 , -0.08632868,  0.00466862, ..., -0.02498488,
        -0.1154283 , -0.04998776],
       [ 0.1461537 , -0.00920811,  0.15359865, ...,  0.03863118,
         0.00484799,  0.11674696],
       ..., 
       [-0.06765957,  0.01211404,  0.04804566, ...,  0.10744154,
        -0.18509543,  0.00261966],
       [-0.14587052, -0.01725268, -0.01762394, ...,  0.14906743,
        -0.19761479, -0.00383327],
       [ 0.18553935,  0.00454816,  0.08183482, ...,  0.08465029,
         0.00993856,  0.03124738]])

We can retreive the vector of individual words

In [6]:
model['dog'].shape

(100,)

In [16]:
model['dog'][:10]

array([ 0.04763608,  0.06465351, -0.05797839,  0.0512316 ,  0.02643282,
       -0.05051252,  0.103004  ,  0.15226485,  0.14635983,  0.01904961])

We can do simple queries to retreive words similar to "socks" based on cosine similarity:

In [17]:
indexes, metrics = model.cosine('socks')
indexes, metrics

(array([14558, 20175, 26345, 34047, 23666, 22883, 29181, 27665, 24110, 15950]),
 array([ 0.82123554,  0.81799021,  0.81437202,  0.81014296,  0.80449623,
         0.80084697,  0.79941468,  0.79667896,  0.79619992,  0.79593892]))

This returned a tuple with 2 items:
1. numpy array with the indexes of the similar words in the vocabulary
2. numpy array with cosine similarity to each word

Its possible to get the words of those indexes

In [18]:
model.vocab[indexes]

array([u'winged', u'hairy', u'haired', u'nosed', u'striped', u'skirt',
       u'pumpkin', u'plum', u'petals', u'goose'], 
      dtype='<U78')

There is a helper function to create a combined response: a numpy [record array](http://docs.scipy.org/doc/numpy/user/basics.rec.html)

In [19]:
model.generate_response(indexes, metrics)

rec.array([(u'winged',  0.82123554), (u'hairy',  0.81799021),
 (u'haired',  0.81437202), (u'nosed',  0.81014296),
 (u'striped',  0.80449623), (u'skirt',  0.80084697),
 (u'pumpkin',  0.79941468), (u'plum',  0.79667896),
 (u'petals',  0.79619992), (u'goose',  0.79593892)], 
          dtype=[(u'word', '<U78'), (u'metric', '<f8')])

Is easy to make that numpy array a pure python response:

In [11]:
model.generate_response(indexes, metrics).tolist()

[(u'hairy', 0.8375353970603848),
 (u'pumpkin', 0.8359084628493809),
 (u'gravy', 0.8281874915608026),
 (u'nosed', 0.8253361379785071),
 (u'plum', 0.8227839904046932),
 (u'winged', 0.8147638561412592),
 (u'bock', 0.8139092031538545),
 (u'petals', 0.8125379796045767),
 (u'biscuits', 0.8105933044655644),
 (u'striped', 0.8085017054444408)]

### Phrases

Since we trained the model with the output of `word2phrase` we can ask for similarity of "phrases"

In [20]:
indexes, metrics = model.cosine('los_angeles')
model.generate_response(indexes, metrics).tolist()

[(u'san_francisco', 0.9041961529044904),
 (u'san_diego', 0.878928874042809),
 (u'miami', 0.8472139709422473),
 (u'las_vegas', 0.8448023749169071),
 (u'seattle', 0.8300607114425045),
 (u'chicago_illinois', 0.8156216043695941),
 (u'atlanta', 0.8154670462852056),
 (u'chicago', 0.8152084361348005),
 (u'dallas', 0.8133107082046103),
 (u'cincinnati', 0.8131563079370175)]

### Analogies

Its possible to do more complex queries like analogies such as: `king - man + woman = queen` 
This method returns the same as `cosine` the indexes of the words in the vocab and the metric

In [13]:
indexes, metrics = model.analogy(pos=['king', 'woman'], neg=['man'], n=10)
indexes, metrics

(array([1087, 1145, 7523, 3141, 6768, 1335, 8419, 1826,  648, 1426]),
 array([ 0.2917969 ,  0.27353295,  0.26877692,  0.26596514,  0.26487509,
         0.26428581,  0.26315492,  0.26261258,  0.26136635,  0.26099078]))

In [14]:
model.generate_response(indexes, metrics).tolist()

[(u'queen', 0.2917968955611075),
 (u'prince', 0.27353295205311695),
 (u'empress', 0.2687769174818083),
 (u'monarch', 0.2659651399832089),
 (u'regent', 0.26487508713026797),
 (u'wife', 0.2642858109968327),
 (u'aragon', 0.2631549214361766),
 (u'throne', 0.26261257728511833),
 (u'emperor', 0.2613663460665488),
 (u'bishop', 0.26099078142148696)]

### Clusters

In [23]:
clusters = word2vec.load_clusters('./text8-clusters.txt')

We can see get the cluster number for individual words

In [24]:
clusters['dog']

98

We can see get all the words grouped on an specific cluster

In [25]:
clusters.get_words_on_cluster(90).shape

(282,)

In [26]:
clusters.get_words_on_cluster(90)[:10]

array(['based', 'common', 'important', 'making', 'complex', 'simple',
       'difficult', 'active', 'alternative', 'advanced'], dtype=object)

We can add the clusters to the word2vec model and generate a response that includes the clusters

In [27]:
model.clusters = clusters

In [28]:
indexes, metrics = model.analogy(pos=['paris', 'germany'], neg=['france'], n=10)

In [29]:
model.generate_response(indexes, metrics).tolist()

[(u'berlin', 0.3249722453845743, 20),
 (u'munich', 0.2918333854998356, 2),
 (u'vienna', 0.28959643218889897, 82),
 (u'leipzig', 0.277032881247382, 41),
 (u'moscow', 0.2719043091606273, 17),
 (u'st_petersburg', 0.2707043576530187, 63),
 (u'dresden', 0.2582195790748487, 86),
 (u'prague', 0.25692625751562714, 19),
 (u'bonn', 0.25073682763496263, 69),
 (u'z_rich', 0.2496111225441054, 42)]